In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
#tf.debugging.set_log_device_placement(True)
#gpus = tf.config.experimental.list_physical_devices('GPU')
#print(len(gpus))

In [14]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [15]:
#data load, F.txt 파일이 있는 절대 경로를 입력하시면 됩니다.
#data 확인을 위하여 출력해봅니다. column name에 데이터가 들어가있어서 처리해주어야 합니다.
# N = 5
data = pd.read_csv('C:/Users/Administrator/nF1.txt', sep = ',')
data

,-0.035389,-0.227283,0.004185,-0.014511,0.040236,0.076649,0.087099,0.068001,-0.120660,0.127359,...,-0.111942,0.026697,0.021189,0.074564,-0.020004.1,0.194337,-0.000978,-0.137848.1,-0.054925,0.062597
0,-0.002858,0.004370,0.007207,0.000188,0.000679,0.004595,-0.004714,-0.007154,-0.001531,0.006347,...,0.002311,0.000574,-0.002751,-0.002682,-0.004268,0.004539,0.011089,0.005675,-0.004680,-0.009902
1,0.000120,-0.000015,-0.000057,-0.000027,0.000081,-0.000013,0.000028,-0.000000,0.000016,0.000022,...,-0.000052,0.000040,-0.000020,0.000014,-0.000050,0.000013,-0.000036,-0.000046,0.000042,-0.000020
2,0.000314,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000001,0.000000,0.000000,0.000000,0.000000
3,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
arr = []
for i in range(len(data.iloc[0,:])):
    arr.append(i+1)

In [17]:
#load data
#column name에 data가 들어가있기 때문에 "names = arr" 로 처리해 줍니다. arr은 1 ~ len(F)의 숫자가 담겨있습니다.
dataF = pd.read_csv('C:/Users/Administrator/F1.txt', sep = ',', names = arr)
#dataQ : Q로 학습
#dataQ = pd.read_csv('C:/Users/Administrator/Q.txt', sep = ',', names = arr)
dataQ = pd.read_csv('C:/Users/Administrator/hatQ.txt', sep = ',', names = arr)
data_nF = pd.read_csv('C:/Users/Administrator/nF1.txt', sep = ',', names = arr)

In [18]:
dataQ = pd.read_csv('C:/Users/Administrator/hatQ.txt', sep = ',', names = arr)

In [19]:
def make_arr(A):
    arA = []
    for j in range(len(A.iloc[0, :])):
        arA1 = []
        for i in range(len(A.iloc[:,0])):
            tmpA = A.iloc[:,j][i]
            arA1.append(tmpA)
        arA.append(arA1)
    return arA

In [20]:
#make array
arrF = make_arr(dataF)
dataF = np.array(arrF)

arrQ = make_arr(dataQ)
dataQ = np.array(arrQ)

arr_nF = make_arr(data_nF)
data_nF = np.array(arr_nF)

In [21]:
# data 분할

import math
train_size = math.floor(len(dataF)*0.6) # train : 60%
val_size = math.floor(len(dataF)*0.2) #val : 20%
test_size = math.floor(len(dataF)*0.2) #test : 20%
#generate F_data, F_val, F_test
F_data = dataF[:train_size, :]
F_val = dataF[train_size:(val_size + train_size), :]
F_test = dataF[(val_size + train_size):(val_size + train_size + test_size), :]

#generate Q_data, Q_val, Q_test
Q_data = dataQ[:train_size, :]
Q_val = dataQ[train_size:(val_size + train_size), :]
Q_test = dataQ[(val_size + train_size):(val_size + train_size + test_size), :]

#generate nF_data, nF_val, nF_test
nF_data = data_nF[:train_size, :]
nF_val = data_nF[train_size:(val_size + train_size), :]
nF_test = data_nF[(val_size + train_size):(val_size + train_size + test_size), :]

print(f'{F_data.shape}, {F_test.shape}, {F_val.shape}')
print(f'{Q_data.shape}, {Q_test.shape}, {Q_val.shape}')
print(f'{nF_data.shape}, {nF_test.shape}, {nF_val.shape}')

(60000, 5), (20000, 5), (20000, 5)
(60000, 5), (20000, 5), (20000, 5)
(60000, 5), (20000, 5), (20000, 5)


In [22]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential() #Sequential

model.add(keras.layers.Dense(input_dim = 5, units = 10, activation= "tanh",kernel_initializer ="HeNormal" )) 
model.add(keras.layers.Dense(20, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))  
model.add(keras.layers.Dense(100, activation= "tanh")) 
model.add(keras.layers.Dense(120, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))
model.add(keras.layers.Dense(5, activation= "tanh"))

In [23]:
with tf.device('/GPU:0'):
    F_train = tf.constant(F_data)
    Q_train = tf.constant(Q_data)
    nF_train = tf.constant(nF_data)
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

In [24]:
start_vect=time.time()

model.compile(loss = "mse",
              optimizer = "ADAM",
              metrics = ["accuracy"])
history = model.fit(x = F_train, y = Q_train, validation_data=(F_val, Q_val),epochs = 1000)


print("training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

Epoch 1/1000
1875/1875 [==============================] - 1s 491us/step - loss: 9.9307e-05 - accuracy: 0.8138 - val_loss: 4.4116e-05 - val_accuracy: 0.8773
Epoch 2/1000
1875/1875 [==============================] - 1s 463us/step - loss: 5.3929e-05 - accuracy: 0.8248 - val_loss: 5.2042e-05 - val_accuracy: 0.8620
Epoch 3/1000
1875/1875 [==============================] - 1s 466us/step - loss: 5.3138e-05 - accuracy: 0.8267 - val_loss: 4.5485e-05 - val_accuracy: 0.8514
Epoch 4/1000
1875/1875 [==============================] - 1s 465us/step - loss: 5.3170e-05 - accuracy: 0.8261 - val_loss: 5.9030e-05 - val_accuracy: 0.8511
Epoch 5/1000
1875/1875 [==============================] - 1s 464us/step - loss: 5.1126e-05 - accuracy: 0.8279 - val_loss: 5.5044e-05 - val_accuracy: 0.8540
Epoch 6/1000
1875/1875 [==============================] - 1s 466us/step - loss: 4.9981e-05 - accuracy: 0.8281 - val_loss: 3.9795e-05 - val_accuracy: 0.8591
Epoch 7/1000
1875/1875 [==============================] - 1s 466

1875/1875 [==============================] - 1s 472us/step - loss: 4.3427e-05 - accuracy: 0.8336 - val_loss: 4.5424e-05 - val_accuracy: 0.7692
Epoch 106/1000
1875/1875 [==============================] - 1s 467us/step - loss: 4.4609e-05 - accuracy: 0.8312 - val_loss: 5.4702e-05 - val_accuracy: 0.8015
Epoch 107/1000
1875/1875 [==============================] - 1s 472us/step - loss: 4.5065e-05 - accuracy: 0.8290 - val_loss: 6.2776e-05 - val_accuracy: 0.8494
Epoch 108/1000
1875/1875 [==============================] - 1s 477us/step - loss: 4.3803e-05 - accuracy: 0.8313 - val_loss: 3.8989e-05 - val_accuracy: 0.8525
Epoch 109/1000
1875/1875 [==============================] - 1s 477us/step - loss: 4.5269e-05 - accuracy: 0.8309 - val_loss: 3.9890e-05 - val_accuracy: 0.8809
Epoch 110/1000
1875/1875 [==============================] - 1s 466us/step - loss: 4.3515e-05 - accuracy: 0.8306 - val_loss: 3.6889e-05 - val_accuracy: 0.7721
Epoch 111/1000
1875/1875 [==============================] - 1s 465u

Epoch 157/1000
1875/1875 [==============================] - 1s 475us/step - loss: 4.3323e-05 - accuracy: 0.8233 - val_loss: 4.1033e-05 - val_accuracy: 0.7839
Epoch 158/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.4187e-05 - accuracy: 0.8249 - val_loss: 4.7285e-05 - val_accuracy: 0.7949
Epoch 159/1000
1875/1875 [==============================] - 1s 463us/step - loss: 4.2942e-05 - accuracy: 0.8232 - val_loss: 3.8631e-05 - val_accuracy: 0.8346
Epoch 160/1000
1875/1875 [==============================] - 1s 469us/step - loss: 4.3277e-05 - accuracy: 0.8272 - val_loss: 4.8024e-05 - val_accuracy: 0.8375
Epoch 161/1000
1875/1875 [==============================] - 1s 459us/step - loss: 4.3754e-05 - accuracy: 0.8265 - val_loss: 3.5604e-05 - val_accuracy: 0.8393
Epoch 162/1000
1875/1875 [==============================] - 1s 461us/step - loss: 4.3108e-05 - accuracy: 0.8279 - val_loss: 3.5813e-05 - val_accuracy: 0.8839
Epoch 163/1000
1875/1875 [==========================

Epoch 209/1000
1875/1875 [==============================] - 1s 475us/step - loss: 4.1945e-05 - accuracy: 0.8309 - val_loss: 4.1372e-05 - val_accuracy: 0.8696
Epoch 210/1000
1875/1875 [==============================] - 1s 467us/step - loss: 4.3403e-05 - accuracy: 0.8237 - val_loss: 3.8163e-05 - val_accuracy: 0.7431
Epoch 211/1000
1875/1875 [==============================] - 1s 472us/step - loss: 4.2586e-05 - accuracy: 0.8249 - val_loss: 4.3463e-05 - val_accuracy: 0.7442
Epoch 212/1000
1875/1875 [==============================] - 1s 472us/step - loss: 4.3089e-05 - accuracy: 0.8302 - val_loss: 5.3583e-05 - val_accuracy: 0.8332
Epoch 213/1000
1875/1875 [==============================] - 1s 460us/step - loss: 4.3187e-05 - accuracy: 0.8294 - val_loss: 4.5419e-05 - val_accuracy: 0.8526
Epoch 214/1000
1875/1875 [==============================] - 1s 460us/step - loss: 4.2853e-05 - accuracy: 0.8253 - val_loss: 4.8509e-05 - val_accuracy: 0.7596
Epoch 215/1000
1875/1875 [==========================

Epoch 261/1000
1875/1875 [==============================] - 1s 457us/step - loss: 4.2988e-05 - accuracy: 0.8269 - val_loss: 3.6979e-05 - val_accuracy: 0.8561
Epoch 262/1000
1875/1875 [==============================] - 1s 458us/step - loss: 4.3483e-05 - accuracy: 0.8215 - val_loss: 3.7755e-05 - val_accuracy: 0.7444
Epoch 263/1000
1875/1875 [==============================] - 1s 460us/step - loss: 4.2001e-05 - accuracy: 0.8292 - val_loss: 5.0922e-05 - val_accuracy: 0.7415
Epoch 264/1000
1875/1875 [==============================] - 1s 460us/step - loss: 4.3618e-05 - accuracy: 0.8218 - val_loss: 4.1965e-05 - val_accuracy: 0.8306
Epoch 265/1000
1875/1875 [==============================] - 1s 459us/step - loss: 4.2382e-05 - accuracy: 0.8199 - val_loss: 6.0332e-05 - val_accuracy: 0.8758
Epoch 266/1000
1875/1875 [==============================] - 1s 459us/step - loss: 4.2660e-05 - accuracy: 0.8238 - val_loss: 6.0652e-05 - val_accuracy: 0.8723
Epoch 267/1000
1875/1875 [==========================

...

Epoch 781/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.1861e-05 - accuracy: 0.8293 - val_loss: 3.9627e-05 - val_accuracy: 0.8611
Epoch 782/1000
1875/1875 [==============================] - 1s 466us/step - loss: 4.2336e-05 - accuracy: 0.8314 - val_loss: 4.1608e-05 - val_accuracy: 0.7546
Epoch 783/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.1787e-05 - accuracy: 0.8368 - val_loss: 3.9859e-05 - val_accuracy: 0.8849
Epoch 784/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.1852e-05 - accuracy: 0.8329 - val_loss: 5.3124e-05 - val_accuracy: 0.8773
Epoch 785/1000
1875/1875 [==============================] - 1s 462us/step - loss: 4.1485e-05 - accuracy: 0.8295 - val_loss: 5.7381e-05 - val_accuracy: 0.8704
Epoch 786/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.2410e-05 - accuracy: 0.8314 - val_loss: 3.8954e-05 - val_accuracy: 0.8527
Epoch 787/1000
1875/1875 [==========================

Epoch 833/1000
1875/1875 [==============================] - 1s 463us/step - loss: 4.2999e-05 - accuracy: 0.8348 - val_loss: 4.3059e-05 - val_accuracy: 0.8712
Epoch 834/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.3093e-05 - accuracy: 0.8323 - val_loss: 4.5050e-05 - val_accuracy: 0.8702
Epoch 835/1000
1875/1875 [==============================] - 1s 467us/step - loss: 4.2645e-05 - accuracy: 0.8278 - val_loss: 3.6979e-05 - val_accuracy: 0.8821
Epoch 836/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.2243e-05 - accuracy: 0.8337 - val_loss: 3.7976e-05 - val_accuracy: 0.8622
Epoch 837/1000
1875/1875 [==============================] - 1s 466us/step - loss: 4.2464e-05 - accuracy: 0.8306 - val_loss: 4.6884e-05 - val_accuracy: 0.8324
Epoch 838/1000
1875/1875 [==============================] - 1s 461us/step - loss: 4.2210e-05 - accuracy: 0.8345 - val_loss: 4.4916e-05 - val_accuracy: 0.8794
Epoch 839/1000
1875/1875 [==========================

Epoch 885/1000
1875/1875 [==============================] - 1s 473us/step - loss: 4.1426e-05 - accuracy: 0.8304 - val_loss: 4.0191e-05 - val_accuracy: 0.8857
Epoch 886/1000
1875/1875 [==============================] - 1s 490us/step - loss: 4.1900e-05 - accuracy: 0.8262 - val_loss: 3.6671e-05 - val_accuracy: 0.8614
Epoch 887/1000
1875/1875 [==============================] - 1s 464us/step - loss: 4.2713e-05 - accuracy: 0.8324 - val_loss: 4.8602e-05 - val_accuracy: 0.8522
Epoch 888/1000
1875/1875 [==============================] - 1s 477us/step - loss: 4.1651e-05 - accuracy: 0.8298 - val_loss: 3.9599e-05 - val_accuracy: 0.8833
Epoch 889/1000
1875/1875 [==============================] - 1s 466us/step - loss: 4.1784e-05 - accuracy: 0.8328 - val_loss: 3.7814e-05 - val_accuracy: 0.7447
Epoch 890/1000
1875/1875 [==============================] - 1s 462us/step - loss: 4.1680e-05 - accuracy: 0.8296 - val_loss: 3.9724e-05 - val_accuracy: 0.8853
Epoch 891/1000
1875/1875 [==========================

Epoch 937/1000
1875/1875 [==============================] - 1s 471us/step - loss: 4.2303e-05 - accuracy: 0.8277 - val_loss: 4.5361e-05 - val_accuracy: 0.7405
Epoch 938/1000
1875/1875 [==============================] - 1s 505us/step - loss: 4.2244e-05 - accuracy: 0.8307 - val_loss: 4.5353e-05 - val_accuracy: 0.7983
Epoch 939/1000
1875/1875 [==============================] - 1s 481us/step - loss: 4.1676e-05 - accuracy: 0.8265 - val_loss: 3.5253e-05 - val_accuracy: 0.8153
Epoch 940/1000
1875/1875 [==============================] - 1s 483us/step - loss: 4.2386e-05 - accuracy: 0.8311 - val_loss: 4.4742e-05 - val_accuracy: 0.7411
Epoch 941/1000
1875/1875 [==============================] - 1s 467us/step - loss: 4.1794e-05 - accuracy: 0.8300 - val_loss: 6.7529e-05 - val_accuracy: 0.8481
Epoch 942/1000
1875/1875 [==============================] - 1s 462us/step - loss: 4.1782e-05 - accuracy: 0.8316 - val_loss: 5.1930e-05 - val_accuracy: 0.8766
Epoch 943/1000
1875/1875 [==========================

Epoch 989/1000
1875/1875 [==============================] - 1s 458us/step - loss: 4.1671e-05 - accuracy: 0.8284 - val_loss: 3.8955e-05 - val_accuracy: 0.8468
Epoch 990/1000
1875/1875 [==============================] - 1s 458us/step - loss: 4.2135e-05 - accuracy: 0.8273 - val_loss: 4.6776e-05 - val_accuracy: 0.8863
Epoch 991/1000
1875/1875 [==============================] - 1s 458us/step - loss: 4.2256e-05 - accuracy: 0.8334 - val_loss: 3.5746e-05 - val_accuracy: 0.7847
Epoch 992/1000
1875/1875 [==============================] - 1s 457us/step - loss: 4.2055e-05 - accuracy: 0.8271 - val_loss: 3.6935e-05 - val_accuracy: 0.8502
Epoch 993/1000
1875/1875 [==============================] - 1s 457us/step - loss: 4.2043e-05 - accuracy: 0.8309 - val_loss: 3.6220e-05 - val_accuracy: 0.8284
Epoch 994/1000
1875/1875 [==============================] - 1s 458us/step - loss: 4.2338e-05 - accuracy: 0.8299 - val_loss: 3.8821e-05 - val_accuracy: 0.8578
Epoch 995/1000
1875/1875 [==========================

In [25]:
#F 학습 후 F accuracy
result = model.evaluate(F_test, Q_test)

  1/625 [..............................] - ETA: 0s - loss: 2.9706e-05 - accuracy: 0.8125

W1112 05:44:29.548155 16064 callbacks.py:325] Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0010s). Check your callbacks.


625/625 [==============================] - 0s 250us/step - loss: 4.4803e-05 - accuracy: 0.8022


In [26]:
#F 학습 후 nF accuracy
result = model.evaluate(nF_test, Q_test)

625/625 [==============================] - 0s 251us/step - loss: 4.4803e-05 - accuracy: 0.8022


In [ ]:
model = keras.models.Sequential() #Sequentioal

start_vect=time.time()
model.add(keras.layers.Dense(input_dim = 5, units = 10, activation= "tanh",kernel_initializer ="HeNormal" )) 
model.add(keras.layers.Dense(20, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))  
model.add(keras.layers.Dense(100, activation= "tanh")) 
model.add(keras.layers.Dense(120, activation= "tanh")) 
model.add(keras.layers.Dense(50, activation= "tanh"))
model.add(keras.layers.Dense(5, activation= "tanh"))

model.compile(loss = "mse",
              optimizer = "adam",
              metrics = ["accuracy"])
history = model.fit(x = nF_train, y = Q_train, validation_data=(nF_val, Q_val),epochs = 1000)

print("training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

Epoch 1/1000
1875/1875 [==============================] - 1s 484us/step - loss: 8.8941e-05 - accuracy: 0.8281 - val_loss: 5.5157e-05 - val_accuracy: 0.8247
Epoch 2/1000
1875/1875 [==============================] - 1s 465us/step - loss: 4.2645e-05 - accuracy: 0.8191 - val_loss: 1.4773e-04 - val_accuracy: 0.7788
Epoch 3/1000
1875/1875 [==============================] - 1s 472us/step - loss: 2.0022e-05 - accuracy: 0.8217 - val_loss: 3.4462e-06 - val_accuracy: 0.7532
Epoch 4/1000
1875/1875 [==============================] - 1s 460us/step - loss: 1.3323e-05 - accuracy: 0.8211 - val_loss: 1.4475e-06 - val_accuracy: 0.8590
Epoch 5/1000
1875/1875 [==============================] - 1s 455us/step - loss: 8.6461e-06 - accuracy: 0.8330 - val_loss: 4.2563e-06 - val_accuracy: 0.8184
Epoch 6/1000
1875/1875 [==============================] - 1s 452us/step - loss: 8.3436e-06 - accuracy: 0.8305 - val_loss: 1.3358e-05 - val_accuracy: 0.8517
Epoch 7/1000
1875/1875 [==============================] - 1s 453

1875/1875 [==============================] - 1s 469us/step - loss: 1.2920e-06 - accuracy: 0.8373 - val_loss: 8.4786e-07 - val_accuracy: 0.8771
Epoch 106/1000
1875/1875 [==============================] - 1s 458us/step - loss: 1.3274e-06 - accuracy: 0.8328 - val_loss: 8.4176e-07 - val_accuracy: 0.8188
Epoch 107/1000
1875/1875 [==============================] - 1s 456us/step - loss: 1.1856e-06 - accuracy: 0.8378 - val_loss: 8.7448e-07 - val_accuracy: 0.8155
Epoch 108/1000
1875/1875 [==============================] - 1s 465us/step - loss: 1.0710e-06 - accuracy: 0.8324 - val_loss: 2.8338e-07 - val_accuracy: 0.7578
Epoch 109/1000
1875/1875 [==============================] - 1s 473us/step - loss: 1.5238e-06 - accuracy: 0.8349 - val_loss: 1.9179e-06 - val_accuracy: 0.8814
Epoch 110/1000
1875/1875 [==============================] - 1s 468us/step - loss: 1.1355e-06 - accuracy: 0.8334 - val_loss: 1.9399e-06 - val_accuracy: 0.7954
Epoch 111/1000
1875/1875 [==============================] - 1s 469u

Epoch 157/1000
1875/1875 [==============================] - 1s 474us/step - loss: 8.5090e-07 - accuracy: 0.8345 - val_loss: 8.7950e-07 - val_accuracy: 0.8704
Epoch 158/1000
1875/1875 [==============================] - 1s 471us/step - loss: 9.7594e-07 - accuracy: 0.8344 - val_loss: 3.1641e-06 - val_accuracy: 0.8662
Epoch 159/1000
1875/1875 [==============================] - 1s 501us/step - loss: 9.7017e-07 - accuracy: 0.8318 - val_loss: 5.5163e-07 - val_accuracy: 0.8583
Epoch 160/1000
1875/1875 [==============================] - 1s 489us/step - loss: 1.0193e-06 - accuracy: 0.8325 - val_loss: 6.0734e-07 - val_accuracy: 0.8737
Epoch 161/1000
1875/1875 [==============================] - 1s 496us/step - loss: 1.0203e-06 - accuracy: 0.8361 - val_loss: 7.0228e-07 - val_accuracy: 0.7599
Epoch 162/1000
1875/1875 [==============================] - 1s 485us/step - loss: 9.5467e-07 - accuracy: 0.8312 - val_loss: 1.6620e-06 - val_accuracy: 0.8827
Epoch 163/1000
1875/1875 [==========================

Epoch 209/1000
1875/1875 [==============================] - 1s 460us/step - loss: 8.4556e-07 - accuracy: 0.8331 - val_loss: 4.8206e-07 - val_accuracy: 0.8490
Epoch 210/1000
1875/1875 [==============================] - 1s 459us/step - loss: 5.5207e-07 - accuracy: 0.8342 - val_loss: 3.1260e-06 - val_accuracy: 0.8391
Epoch 211/1000
1875/1875 [==============================] - 1s 461us/step - loss: 7.5018e-07 - accuracy: 0.8283 - val_loss: 2.2752e-07 - val_accuracy: 0.8331
Epoch 212/1000
1875/1875 [==============================] - 1s 460us/step - loss: 5.4499e-07 - accuracy: 0.8348 - val_loss: 1.7124e-07 - val_accuracy: 0.8835
Epoch 213/1000
1875/1875 [==============================] - 1s 462us/step - loss: 7.3837e-07 - accuracy: 0.8346 - val_loss: 1.8736e-07 - val_accuracy: 0.8859
Epoch 214/1000
1875/1875 [==============================] - 1s 461us/step - loss: 7.7477e-07 - accuracy: 0.8375 - val_loss: 8.4506e-07 - val_accuracy: 0.8567
Epoch 215/1000
1875/1875 [==========================

Epoch 261/1000
1875/1875 [==============================] - 1s 460us/step - loss: 6.7320e-07 - accuracy: 0.8281 - val_loss: 3.1956e-07 - val_accuracy: 0.8579
Epoch 262/1000
1875/1875 [==============================] - 1s 463us/step - loss: 5.2800e-07 - accuracy: 0.8224 - val_loss: 1.4865e-07 - val_accuracy: 0.8809
Epoch 263/1000
1875/1875 [==============================] - 1s 460us/step - loss: 5.7049e-07 - accuracy: 0.8253 - val_loss: 1.5975e-07 - val_accuracy: 0.7470
Epoch 264/1000
1875/1875 [==============================] - 1s 460us/step - loss: 6.7165e-07 - accuracy: 0.8311 - val_loss: 9.7864e-07 - val_accuracy: 0.7738
Epoch 265/1000
1875/1875 [==============================] - 1s 463us/step - loss: 5.0929e-07 - accuracy: 0.8310 - val_loss: 4.0076e-07 - val_accuracy: 0.8461
Epoch 266/1000
1875/1875 [==============================] - 1s 460us/step - loss: 5.5280e-07 - accuracy: 0.8281 - val_loss: 3.9653e-07 - val_accuracy: 0.7613
Epoch 267/1000
1875/1875 [==========================

Epoch 313/1000
1875/1875 [==============================] - 1s 461us/step - loss: 4.9668e-07 - accuracy: 0.8268 - val_loss: 1.8286e-06 - val_accuracy: 0.8658
Epoch 314/1000
1875/1875 [==============================] - 1s 464us/step - loss: 5.1001e-07 - accuracy: 0.8271 - val_loss: 6.6736e-07 - val_accuracy: 0.8580
Epoch 315/1000
1875/1875 [==============================] - 1s 464us/step - loss: 6.2391e-07 - accuracy: 0.8255 - val_loss: 5.6088e-07 - val_accuracy: 0.7521
Epoch 316/1000
1875/1875 [==============================] - 1s 461us/step - loss: 5.7727e-07 - accuracy: 0.8251 - val_loss: 2.9337e-07 - val_accuracy: 0.7451
Epoch 317/1000
1875/1875 [==============================] - 1s 463us/step - loss: 4.6394e-07 - accuracy: 0.8258 - val_loss: 1.2651e-07 - val_accuracy: 0.8590
Epoch 318/1000
1875/1875 [==============================] - 1s 461us/step - loss: 5.2900e-07 - accuracy: 0.8287 - val_loss: 3.9221e-07 - val_accuracy: 0.7976
Epoch 319/1000
1875/1875 [==========================

In [ ]:
#nF 학습 후 nF accuracy
result = model.evaluate(nF_test, Q_test)